In [ ]:
import sys
import os

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running locally or in Binder")

# Install UppASD if in Colab
if IN_COLAB:
    BRANCH_NAME = "python26"
    
    print(f"Installing UppASD from GitHub (branch: {BRANCH_NAME})...")
    print("This will take approximately 2-3 minutes...")
    
    # Clone repository
    !git clone --branch {BRANCH_NAME} https://github.com/UppASD/UppASD.git /content/UppASD
    
    # Install (prefer OpenBLAS on Colab). Use non-editable install so Colab imports work.
    %cd /content/UppASD
    !BLA_VENDOR=OpenBLAS FC=gfortran pip install . --quiet
    # Return to content directory
    %cd /content
    
    print("✓ Installation complete!")
    print("✓ UppASD is ready to use")

## Cloud Environment Setup

This cell detects if you're running on Google Colab and installs UppASD if needed. It does nothing when running locally or on Binder (where UppASD is pre-installed).

# Multi-Species UppASD Interactive Setup
Create and run atomistic spin dynamics simulations with multiple atom types (Fe, Co, B).

This notebook demonstrates:
1. Define crystal structure with multiple chemical species
2. Set up species-dependent magnetic moments
3. Configure type-resolved exchange interactions
4. Run time evolution dynamics
5. Plot magnetization and energy vs time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

%matplotlib inline

## 1. Define Multi-Species Crystal Structure
Create a layered Fe/Co/B system with three atom types.

In [ ]:
# Lattice parameters (simple cubic for demonstration)
alat = 2.5  # Angstrom (lattice scaling factor)

# Lattice vectors (normalized Cartesian unit vectors)
# UppASD uses these as basis, scaled by 'cell' in inpsd.dat
lattice_vectors = np.array([
    [1.0, 0.0, 0.0],  # a1
    [0.0, 1.0, 0.0],  # a2
    [0.0, 0.0, 1.0]   # a3
])

# Basis atoms (fractional coordinates) with atom types
# Format: [x, y, z, type] where type: 1=Fe, 2=Co, 3=B
basis_fractional = np.array([
    [0.0, 0.0, 0.0, 1],  # Fe at corner
    [0.5, 0.5, 0.0, 2],  # Co in-plane center
    [0.5, 0.0, 0.5, 2],  # Co 
    [0.0, 0.5, 0.5, 3],  # B
])

# System size (supercell repetitions)
n1, n2, n3 = 8, 8, 4
natom = len(basis_fractional) * n1 * n2 * n3

print(f"System: {natom} atoms in {n1}×{n2}×{n3} supercell")
print(f"Lattice constant: {alat} Å")
print(f"Basis atoms: {len(basis_fractional)}")
print(f"  Fe atoms: {np.sum(basis_fractional[:, 3] == 1)} per cell")
print(f"  Co atoms: {np.sum(basis_fractional[:, 3] == 2)} per cell")
print(f"  B atoms: {np.sum(basis_fractional[:, 3] == 3)} per cell")

## 2. Generate Atomic Coordinates with Types
Build full supercell preserving atom type information.

In [ ]:
# Generate coordinates using helper function
from uppasd import notebook as nb

coords, atom_types = nb.generate_supercell_coordinates(
    lattice_vectors, basis_fractional, n1, n2, n3, scale=alat
)

print(f"Generated {len(coords)} coordinates")
print(f"Atom type distribution:")
print(f"  Type 1 (Fe): {np.sum(atom_types == 1)} atoms")
print(f"  Type 2 (Co): {np.sum(atom_types == 2)} atoms")
print(f"  Type 3 (B):  {np.sum(atom_types == 3)} atoms")
print(f"\nFirst 5 atoms:")
for i in range(5):
    print(f"  {i + 1}: type={atom_types[i]}, pos={coords[i]}")

## 3. Define Species-Dependent Magnetic Moments
Set different moment magnitudes for Fe, Co, and B.

In [ ]:
# Magnetic moment magnitudes per species (μB)
moments_per_type = {
    1: 2.2,  # Fe
    2: 1.7,  # Co
    3: 0.5   # B (small moment)
}

# Initialize ferromagnetic moments and add perturbation
moments = nb.create_ferromagnetic_moments(natom, moment_magnitude=1.0)

# Apply species-dependent magnitudes and add perturbation
for i, atype in enumerate(atom_types):
    moments[i] *= moments_per_type[atype]

moments = nb.perturb_moments(
    moments, 
    perturbation_scale=0.01, 
    seed=42, 
    renormalize=True,
    moment_magnitudes=np.array([moments_per_type[t] for t in atom_types])
)

print(f"Moments shape: {moments.shape}")
print(f"Mean moment per type:")
for atype in [1, 2, 3]:
    mask = atom_types == atype
    mean_mag = np.mean(np.linalg.norm(moments[mask], axis=1))
    print(f"  Type {atype}: {mean_mag:.3f} μB")

## 4. Define Type-Resolved Exchange Interactions
Set up different J values for Fe-Fe, Fe-Co, Co-Co, Co-B, etc.

In [ ]:
# Exchange parameters (mRy = milli-Rydberg)
# Note: 1 meV ≈ 0.0735 mRy, 1 mRy ≈ 13.6 meV
# 
# Format: {(type_i, type_j, distance_shell): J_value}
# distance_shell: 1=NN (nearest neighbor), 2=NNN (next-nearest), etc.
# 
# This allows different J values for different coordination shells
exchange_matrix = {
    # Nearest neighbor interactions (shell 1)
    (1, 1, 1): 1.544,   # Fe-Fe NN (~21 meV)
    (2, 2, 1): 1.323,   # Co-Co NN (~18 meV)
    (3, 3, 1): 0.368,   # B-B NN (~5 meV)
    (1, 2, 1): 1.433,   # Fe-Co NN (~19.5 meV)
    (2, 1, 1): 1.433,   # Co-Fe NN (symmetric)
    (1, 3, 1): 0.588,   # Fe-B NN (~8 meV)
    (3, 1, 1): 0.588,   # B-Fe NN
    (2, 3, 1): 0.662,   # Co-B NN (~9 meV)
    (3, 2, 1): 0.662,   # B-Co NN
    
    # Next-nearest neighbor interactions (shell 2) - optional, smaller values
    (1, 1, 2): 0.5,     # Fe-Fe NNN
    (2, 2, 2): 0.4,     # Co-Co NNN
    (1, 2, 2): 0.45,    # Fe-Co NNN
    (2, 1, 2): 0.45,    # Co-Fe NNN
}

# Define distance ranges for each shell (in units of alat)
# These define which neighbors belong to which shell
distance_shells = {
    1: (0.0, 0.9),      # NN: 0 < r < 0.9*alat (catches vectors like [0.5,0.5,0])
    2: (0.9, 1.3),      # NNN: 0.9*alat < r < 1.3*alat (catches vectors like [1,0,0])
    3: (1.3, 2.0),      # Further neighbors
}

print("Exchange interaction matrix (mRy):")
print("\nNearest Neighbors (NN):")
print("     Fe    Co    B")
for i in [1, 2, 3]:
    row = []
    for j in [1, 2, 3]:
        val = exchange_matrix.get((i, j, 1), 0.0)
        row.append(f"{val:5.3f}")
    label = ['Fe', 'Co', 'B'][i-1]
    print(f"{label}  " + "  ".join(row))

print("\nNext-Nearest Neighbors (NNN):")
print("     Fe    Co    B")
for i in [1, 2, 3]:
    row = []
    for j in [1, 2, 3]:
        val = exchange_matrix.get((i, j, 2), 0.0)
        row.append(f"{val:5.3f}")
    label = ['Fe', 'Co', 'B'][i-1]
    print(f"{label}  " + "  ".join(row))

## 5. Build Exchange Neighbor List
Use ASE to automatically find neighbors and compute exchange vectors.

The exchange matrix defined above uses a **shell-based** approach:
- Shell 1: Nearest neighbors (NN) - e.g., Fe-Co at (0.5, 0.5, 0.0)
- Shell 2: Next-nearest neighbors (NNN) - e.g., Fe-Fe at (1.0, 0.0, 0.0)
- etc.

The `build_exchange_list` function will:
1. Find all neighbors within cutoff using ASE
2. Calculate distance |r_ij| for each pair
3. Assign to appropriate shell based on distance
4. Look up J_ij value from exchange_matrix
5. Return exchange vectors in fractional coordinates

In [ ]:
from ase import Atoms
from ase.neighborlist import neighbor_list

def build_exchange_list(lattice_vectors, basis_positions, basis_types, 
                        exchange_matrix, distance_shells, cutoff=3.0, scale=1.0):
    """
    Build exchange interaction list using ASE neighbor finder.
    
    This function automatically generates ALL symmetry-equivalent exchange vectors.
    For example, if Fe-Co NN exists at (0.5, 0.5, 0.0), it will also find:
    (-0.5, -0.5, 0.0), (0.5, -0.5, 0.0), (-0.5, 0.5, 0.0), etc.
    
    Parameters:
    -----------
    lattice_vectors : array (3, 3)
        Normalized lattice vectors (will be scaled by 'scale')
    basis_positions : array (n_basis, 3)
        Fractional coordinates of basis atoms
    basis_types : array (n_basis,)
        Atom types for each basis atom
    exchange_matrix : dict
        Exchange parameters J_ij for each (type_i, type_j, shell) tuple in mRy
        Format: {(itype, jtype, shell_index): J_value}
    distance_shells : dict
        Distance ranges for each shell: {shell: (min_dist, max_dist)}
        Distances in units of 'scale' (alat)
    cutoff : float
        Neighbor search cutoff distance in Angstroms
    scale : float
        Lattice constant (alat) to scale lattice vectors
        
    Returns:
    --------
    exchange_list : list of tuples
        Each tuple: (type_i, type_j, r_ij_x, r_ij_y, r_ij_z, J_ij)
        r_ij in fractional coordinates (units of alat)
        Includes ALL symmetry-equivalent vectors
    """
    # Create ASE Atoms object for unit cell
    cell_scaled = lattice_vectors * scale
    atoms = Atoms(
        numbers=basis_types,  # Use type as atomic number temporarily
        positions=basis_positions @ cell_scaled,  # Convert to Cartesian
        cell=cell_scaled,
        pbc=True
    )
    
    # Find all neighbors within cutoff
    i_indices, j_indices, offsets = neighbor_list('ijS', atoms, cutoff)
    
    # Build exchange list with unique displacement vectors
    exchange_dict = {}  # key: (type_i, type_j, r_ij_tuple), value: J_ij
    
    for idx in range(len(i_indices)):
        i = i_indices[idx]
        j = j_indices[idx]
        offset = offsets[idx]
        
        # Get atom types
        type_i = basis_types[i]
        type_j = basis_types[j]
        
        # Calculate displacement vector in Cartesian (Angstroms)
        r_i = atoms.positions[i]
        r_j = atoms.positions[j] + offset @ cell_scaled
        r_ij_cart = r_j - r_i
        
        # Convert to fractional coordinates (in units of alat)
        r_ij_frac = r_ij_cart / scale
        
        # Calculate distance in units of alat
        dist = np.linalg.norm(r_ij_frac)
        
        # Skip self-interaction
        if dist < 1e-6:
            continue
        
        # Determine which shell this distance belongs to
        shell = None
        for s, (d_min, d_max) in distance_shells.items():
            if d_min < dist <= d_max:
                shell = s
                break
        
        if shell is None:
            continue  # Distance not in any defined shell
        
        # Check if this type pair + shell has defined exchange
        if (type_i, type_j, shell) not in exchange_matrix:
            continue
        
        J_ij = exchange_matrix[(type_i, type_j, shell)]
        
        # Round to avoid floating point issues
        r_ij_tuple = tuple(np.round(r_ij_frac, decimals=6))
        
        # Store unique vectors (this includes all symmetry-equivalent ones)
        key = (type_i, type_j, r_ij_tuple)
        if key not in exchange_dict:
            exchange_dict[key] = J_ij
    
    # Convert to list format for jfile
    exchange_list = []
    for (type_i, type_j, r_ij_tuple), J_ij in sorted(exchange_dict.items()):
        exchange_list.append((
            type_i, type_j,
            r_ij_tuple[0], r_ij_tuple[1], r_ij_tuple[2],
            J_ij
        ))
    
    return exchange_list

# Build exchange list
exchange_list = build_exchange_list(
    lattice_vectors, 
    basis_fractional[:, :3], 
    basis_fractional[:, 3].astype(int),
    exchange_matrix,
    distance_shells,
    cutoff=4.0,  # Angstroms (large enough to catch NNN)
    scale=alat
)

print(f"✓ Found {len(exchange_list)} unique exchange vectors")
print("  (includes all symmetry-equivalent interactions)")

# Count interactions by type pair and shell
from collections import defaultdict
interaction_counts = defaultdict(int)
for (ti, tj, rx, ry, rz, J) in exchange_list:
    dist = np.sqrt(rx**2 + ry**2 + rz**2)
    shell = '?'
    for s, (d_min, d_max) in distance_shells.items():
        if d_min < dist <= d_max:
            shell = s
            break
    interaction_counts[(ti, tj, shell)] += 1

print("\nInteraction summary:")
type_labels = {1: 'Fe', 2: 'Co', 3: 'B'}
shell_labels = {1: 'NN', 2: 'NNN', 3: 'NNNN'}
for (ti, tj, shell), count in sorted(interaction_counts.items()):
    print(f"  {type_labels[ti]}-{type_labels[tj]} {shell_labels.get(shell, f'S{shell}')}: "
          f"{count} vectors (symmetry-equivalent)")

print(f"\nFirst 15 exchange interactions:")
for i, (ti, tj, rx, ry, rz, J) in enumerate(exchange_list[:15]):
    dist = np.sqrt(rx**2 + ry**2 + rz**2)
    shell = '?'
    for s, (d_min, d_max) in distance_shells.items():
        if d_min < dist <= d_max:
            shell = s
            break
    shell_label = {1: 'NN', 2: 'NNN', 3: 'NNNN'}.get(shell, f'S{shell}')
    print(f"  {i+1}. {type_labels[ti]}-{type_labels[tj]} ({shell_label}): "
          f"r=({rx:6.3f}, {ry:6.3f}, {rz:6.3f}), |r|={dist:.3f}, J={J:.3f} mRy")

## 6. Write Input Files
Create UppASD input files using the automatically generated exchange list.

In [ ]:
# Create working directory
work_dir = Path('./multi_species_sim')
work_dir.mkdir(exist_ok=True)

# Create simulation configuration using helper
config = nb.create_relaxation_protocol(
    'mc_sd',
    mc_steps=1000, mc_temp=300,
    sd_steps=5000, sd_temp=100, damping=0.05
)

# Add structure information
config.update({
    'simid': 'multi_species',
    'ncell': [n1, n2, n3],
    'cell': np.eye(3),
    'alat': alat,
    'posfile': './posfile.dat',
    'momfile': './momfile.dat',
    'exchange': './jfile.dat',
    'do_prnstruct': 1,
    'Mensemble': 1,
    'Initmag': 3,
    'avrg_step': 10,
})

# Write all input files using helper functions
nb.write_inpsd_file(work_dir / 'inpsd.dat', config)
nb.write_posfile(work_dir / 'posfile.dat', basis_fractional)
nb.write_momfile(work_dir / 'momfile.dat', moments_per_type, 
                 atom_types=basis_fractional[:, 3].astype(int))

print("✓ Created inpsd.dat with MC+SD protocol")
print(f"  alat: {alat} Å, cell: normalized unit vectors")
print(f"✓ Created posfile.dat ({len(basis_fractional)} basis atoms)")
print(f"✓ Created momfile.dat (Fe=2.2, Co=1.7, B=0.5 μB)")

In [ ]:
# Write momfile.dat
# Format: atom_index 1 m_mag m_x m_y m_z
with open(work_dir / 'momfile.dat', 'w') as f:
    for i, atom_data in enumerate(basis_fractional, 1):
        atype = int(atom_data[3])
        mag = moments_per_type[atype]
        f.write(f"{i}  1  {mag:.6f}  0.0 0.0 1.0\n")

print(f"✓ Created momfile.dat")
print(f"  Content preview:")
with open(work_dir / 'momfile.dat', 'r') as f:
    print("  " + f.read())

In [ ]:
# Write exchange interactions using helper
nb.write_jfile(work_dir / 'jfile.dat', exchange_list)

print(f"✓ Created jfile.dat with {len(exchange_list)} interactions")
print(f"\nAll input files ready in: {work_dir}")

## 7. Run Simulation
Execute time evolution and collect trajectory data.

In [ ]:
# Change to working directory
import os
from pathlib import Path
# Ensure work_dir exists (fallback)
if 'work_dir' not in globals():
    work_dir = Path('./relax_sim')
    work_dir.mkdir(exist_ok=True)

original_dir = os.getcwd()
try:
    os.chdir(work_dir)
except Exception as e:
    print(f"Could not switch to work_dir {work_dir}: {e}")
    work_dir = Path(original_dir)
    os.chdir(original_dir)

try:
    from uppasd import Simulator
    
    # Storage for trajectory (will contain final state when running without callback)
    trajectory = {
        'time': [],
        'energy': [],
        'magnetization': [],
        'mag_per_type': {1: [], 2: [], 3: []},
    }

    def record_timestep(step, moments, energy):
        """Callback to record trajectory data (kept for interactive use)."""
        if step % 10 == 0:  # Record every 10 steps
            trajectory['time'].append(step)
            trajectory['energy'].append(energy)
            
            # Total magnetization
            mag = np.mean(moments, axis=(1, 2))
            mag_norm = np.linalg.norm(mag)
            trajectory['magnetization'].append(mag_norm)
            
            # Per-type magnetization
            moments_reshaped = moments[:, :, 0]  # (3, natom)
            for atype in [1, 2, 3]:
                mask = atom_types == atype
                mag_type = np.mean(moments_reshaped[:, mask], axis=1)
                mag_type_norm = np.linalg.norm(mag_type)
                trajectory['mag_per_type'][atype].append(mag_type_norm)
            
            if step % 500 == 0:
                print(f"Step {step:5d}: E = {energy:10.3f}, |M| = {mag_norm:.3f}")

    print("Initializing simulator...")
    with Simulator() as sim:
        print(f"✓ Initialized: {sim.natom} atoms, {sim.mensemble} ensembles")
        
        # Run time evolution (no Python callback to allow bulk Fortran relax call)
        print("\nRunning spin dynamics (mode=S) without Python callback (faster)...")
        final_moments = sim.relax(
            mode='S',
            temperature=100,
            steps=5000,
            timestep=1e-16,
            damping=0.05,
        )
        
        print(f"\n✓ Simulation complete")
        print(f"✓ Final energy: {sim.energy:.3f}")
        mag_final = np.linalg.norm(np.mean(final_moments, axis=(1, 2)))
        print(f"✓ Final |M|: {mag_final:.3f}")

        # Populate minimal trajectory for plotting (single final point)
        trajectory['time'].append(sim.nstep if hasattr(sim, 'nstep') else 0)
        trajectory['energy'].append(sim.energy)
        trajectory['magnetization'].append(mag_final)
        moments_reshaped = final_moments[:, :, 0]
        for atype in [1, 2, 3]:
            mask = atom_types == atype
            if mask.sum() > 0:
                mag_type = np.mean(moments_reshaped[:, mask], axis=1)
                trajectory['mag_per_type'][atype].append(np.linalg.norm(mag_type))
            else:
                trajectory['mag_per_type'][atype].append(0.0)

except ImportError:
    print("\n⚠️  UppASD Python module not found!")
    print("Install with: pip install -e /path/to/UppASD")
    print("\nAlternatively, run from terminal:")
    print(f"  cd {work_dir}")
    print("  sd < inpsd.dat")
finally:
    os.chdir(original_dir)

## 8. Plot Magnetization vs Time
Visualize total and species-resolved magnetization dynamics.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Convert time steps to physical time (femtoseconds)
time_fs = np.array(trajectory['time']) * 0.1  # timestep = 1e-16 s = 0.1 fs

# Plot 1: Total magnetization
ax1.plot(time_fs, trajectory['magnetization'], 'b-', linewidth=2, label='Total')
ax1.set_xlabel('Time (fs)', fontsize=12)
ax1.set_ylabel('|M| (μB/atom)', fontsize=12)
ax1.set_title('Total Magnetization vs Time', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)

# Plot 2: Species-resolved magnetization
colors = {'Fe': 'red', 'Co': 'blue', 'B': 'green'}
labels = {1: 'Fe', 2: 'Co', 3: 'B'}
for atype in [1, 2, 3]:
    label = labels[atype]
    color = colors[label]
    ax2.plot(time_fs, trajectory['mag_per_type'][atype], 
             color=color, linewidth=2, label=label)

ax2.set_xlabel('Time (fs)', fontsize=12)
ax2.set_ylabel('|M| (μB/atom)', fontsize=12)
ax2.set_title('Species-Resolved Magnetization vs Time', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)

plt.tight_layout()
plt.savefig(work_dir / 'magnetization_vs_time.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {work_dir / 'magnetization_vs_time.png'}")
plt.show()

## 9. Plot Energy vs Time
Visualize energy evolution during relaxation.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(time_fs, trajectory['energy'], 'r-', linewidth=2)
ax.set_xlabel('Time (fs)', fontsize=12)
ax.set_ylabel('Energy (code units)', fontsize=12)
ax.set_title('Total Energy vs Time', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add text with energy statistics
E_initial = trajectory['energy'][0]
E_final = trajectory['energy'][-1]
E_change = E_final - E_initial
textstr = f'Initial E: {E_initial:.2f}\nFinal E: {E_final:.2f}\nΔE: {E_change:.2f}'
ax.text(0.02, 0.98, textstr, transform=ax.transAxes,
        fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig(work_dir / 'energy_vs_time.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {work_dir / 'energy_vs_time.png'}")
plt.show()

## 10. Combined Plot
Show both magnetization and energy on the same figure.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Magnetization
ax1.plot(time_fs, trajectory['magnetization'], 'b-', linewidth=2.5)
ax1.set_xlabel('Time (fs)', fontsize=12)
ax1.set_ylabel('|M| (μB/atom)', fontsize=12)
ax1.set_title('Magnetization Dynamics', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Right: Energy
ax2.plot(time_fs, trajectory['energy'], 'r-', linewidth=2.5)
ax2.set_xlabel('Time (fs)', fontsize=12)
ax2.set_ylabel('Energy (code units)', fontsize=12)
ax2.set_title('Energy Evolution', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.suptitle('Multi-Species Fe-Co-B System Dynamics', 
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(work_dir / 'dynamics_combined.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {work_dir / 'dynamics_combined.png'}")
plt.show()

## Summary

This notebook demonstrated:
- ✓ Multi-species system setup (Fe, Co, B)
- ✓ Species-dependent magnetic moments
- ✓ Type-resolved exchange interactions
- ✓ Time evolution dynamics (mode=S)
- ✓ Magnetization vs time analysis
- ✓ Energy vs time tracking

**Key files created:**
- `posfile.dat`: Multi-type atomic positions
- `momfile.dat`: Species-dependent moments
- `jfile.dat`: Type-resolved exchange couplings
- Plots: magnetization and energy dynamics

**Next steps:**
- Modify exchange matrix for different materials
- Add temperature dependence studies
- Analyze correlation functions
- Compute type-resolved susceptibilities